# Script to cut out parts of screenshots and filter them to keep only cards

It takes screenshots of two sizes which are available to us, finds objects in them and finally filters out cards.

NOTE: sizes are set specifically to screenshots we have right now. Idea for improvement would be reworking filtering code to filter based on sides ratio.

In [ ]:
import cv2 as cv
import numpy as np
import random
import os
import shutil

Following function is going to take in image, change its color spectrum and apply threshold of 170. I've tried different color spectrums and thresholds and these seem to be working the best.

Next, it's going to apply OpenCV findContours function, mask them and cut out of original picture. If the cut out picture is bigger than 65 percentile of all pictrues it's going to be saved in separate folder.

In [ ]:
def cutout_custom_percentile_170(image_name):
    
    image_orig = cv.imread(f'to_cut/our_game/{image_name}')
    
    image_name = image_name[:-4]
    
    x_orig, y_orig, z_orig = image_orig.shape
    
    image = image_orig.copy()
    img_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
    ret, thresh = cv.threshold(img_gray, 170, 255, cv.THRESH_BINARY)
    
    contours, hierarchy = cv.findContours(image=thresh, mode=cv.RETR_TREE, method=cv.CHAIN_APPROX_NONE)
 
    xs = []
    ys = []

    all_n = len(contours)

    randomlist = random.sample(range(0,10000), all_n)

    for i in randomlist:
        idx = randomlist.index(i)
        mask = np.zeros_like(img_gray)
        cv.drawContours(mask, contours, idx, 255, -1)

        res = cv.bitwise_and(image, image, mask = mask)
        res_copy = res.copy()

        (y, x) = np.where(mask==255)
        (topy, topx) = (np.min(y), np.min(x))
        (bottomy, bottomx) = (np.max(y), np.max(x))
        out=res_copy[topy:bottomy+1, topx:bottomx+1]
        x, y = out.shape[:2]
        
        xs.append(x)
        ys.append(y)
        
        if x > (np.percentile(xs, 65)):
            if y > (np.percentile(ys, 65)):
                cv.imwrite(f'test_cutout/{image_name}_cutout_masked_{idx}_170.jpg', out)
#     print("Done, 170!")

In [ ]:
# applying functions

directory = os.fsencode('to_cut/our_game/')

for i in os.listdir(directory):
    filename = os.fsdecode(i)

    if filename.endswith(".jpg") or filename.endswith(".png"):
#         print(filename)

        cutout_custom_percentile_170(filename)

Following function is filtering pictures to keep only cards. We only had two screenshot sizes available, so I manually chose sizes of pictures to keep.

In [ ]:
# this filters pictures to keep cards based on their size

directory = os.fsencode('test_cutout/')

for i in os.listdir(directory):
    filename = os.fsdecode(i)

    if filename.endswith(".jpg") or filename.endswith(".png"):
#         print(filename)

        image = cv.imread(f'test_cutout/{filename}')
        y, x, z = image.shape
#         print(x, y)
        
        if (x >= 100 and x <= 110) or (x == 153) or (x >= 20 and x <= 25):
            if (y >= 130 and y <= 145) or (y >= 48 and y <= 50) or (y >= 56 and y <= 60) or (y >= 70 and y <= 75):
                shutil.copy(f'test_cutout/{filename}', 'test_cutout/dataset')
                
        elif (x >= 180 and x <= 185) or (x >= 40 and x <= 45) or (x >=270 and x <=280):
            if (y >= 245 and y <= 255) or (y >= 115 and y <= 120) or (y >= 140 and y <= 145) or (y >= 80 and y <= 95):
                shutil.copy(f'test_cutout/{filename}', 'test_cutout/dataset')

To improve model results I use following function to pad small images.

In [ ]:
# padding small images

directory = os.fsencode('test_cutout/dataset/')

for i in os.listdir(directory):
    filename = os.fsdecode(i)
    
    if filename.endswith(".jpg"):
        image = cv.imread(f'test_cutout/dataset/{filename}')
        
        image_name = filename[:-4]
        
        y,x,z = image.shape

        if (x >= 15 and x <= 25):
#             print(x,y)
            if (y >= 130 and y <= 140):
                top = 0
                bottom = top
                left = int((100 - x)/2)
                right = left
                filled_image = cv.copyMakeBorder(image, top, bottom, left, right, cv.BORDER_CONSTANT, None, (0,0,0))
                cv.imwrite(f'test_cutout/dataset/{image_name}_filled_image.jpg', filled_image)
                
            elif (y >= 45 and y <= 80):
                top = int((138-y)/2)
                bottom = top
                left = int((100 - x)/2)
                right = left
                filled_image = cv.copyMakeBorder(image, top, bottom, left, right, cv.BORDER_CONSTANT, None, (0,0,0))
                cv.imwrite(f'test_cutout/dataset/{image_name}_filled_image.jpg', filled_image)
                
        elif (x >= 40 and x <= 50):
            if (y >= 240 and y <= 250):
                top = 0
                bottom = top
                left = int((183 - x)/2)
                right = left
                filled_image = cv.copyMakeBorder(image, top, bottom, left, right, cv.BORDER_CONSTANT, None, (0,0,0))
                cv.imwrite(f'test_cutout/dataset/{image_name}_filled_image.jpg', filled_image)
                
            elif (y >= 110 and y <= 120):
                top = int((249-y)/2)
                bottom = top
                left = int((183 - x)/2)
                right = left
                filled_image = cv.copyMakeBorder(image, top, bottom, left, right, cv.BORDER_CONSTANT, None, (0,0,0))
                cv.imwrite(f'test_cutout/dataset/{image_name}_filled_image.jpg', filled_image)
                
            elif (y >= 140 and y <= 150):
                top = int((249-y)/2)
                bottom = top
                left = int((183 - x)/2)
                right = left
                filled_image = cv.copyMakeBorder(image, top, bottom, left, right, cv.BORDER_CONSTANT, None, (0,0,0))
                cv.imwrite(f'test_cutout/dataset/{image_name}_filled_image.jpg', filled_image)
                
            elif (y >= 80 and y <= 95):
                top = int((249-y)/2)
                bottom = top
                left = int((183 - x)/2)
                right = left
                filled_image = cv.copyMakeBorder(image, top, bottom, left, right, cv.BORDER_CONSTANT, None, (0,0,0))
                cv.imwrite(f'test_cutout/dataset/{image_name}_filled_image.jpg', filled_image)